In [41]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [43]:
len(tf.config.list_physical_devices('GPU'))

1

In [44]:
def process_csv_file(file):
    try:
     csv_data = pd.read_csv(file)
    except:
         print("CANT READ CSV: ", file)
         return
    
    if csv_data.isnull().values.any():
            
            return False
    try:
        csv_data = csv_data.drop("class", axis = 1)
    except KeyError:
        pass
    return (csv_data.to_numpy())

In [45]:
!cd /Tmp/ASL-data/Database-augmented/coffee/; ls;
!ls

/bin/bash: line 1: cd: /Tmp/ASL-data/Database-augmented/coffee/: No such file or directory
2_Coding-Competition  ASL-words-translator  ASL-words-translator_old
2_Coding-Competition  ASL-words-translator  ASL-words-translator_old


In [46]:
!cd /Tmp/linxinle/ASL-data/Database-augmented/

In [47]:
# file = "/Tmp/linxinle/ASL-data/Database-augmented/test/coffee/2sGQuduhAf41354.csv"
file = "/media/makishima/DATA1/Personnel/Other learning/Programming/Personal_projects/ASL_Language_translation/training_models/mediapipe/Database-augmented/test/coffee/2sGQuduhAf41354.csv"
process_csv_file(file)

CANT READ CSV:  /media/makishima/DATA1/Personnel/Other learning/Programming/Personal_projects/ASL_Language_translation/training_models/mediapipe/Database-augmented/test/coffee/2sGQuduhAf41354.csv


In [48]:
def set_padding(arr, max_length):
    arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
    return np.expand_dims(arr, 0)

In [49]:
def prepare_data(folder_path, labels, train=True):
    total = 0
    max_length = 211
    
    # find max length
    # for folder in os.listdir(folder_path):
    #     print(folder)
    #     if folder != ".DS_Store":
    #         total += len(os.listdir(os.path.join(folder_path, folder)))
    #         for file in os.listdir(os.path.join(folder_path, folder)):
    #             if file != ".DS_Store":
    #                 data = process_csv_file(os.path.join(folder_path, folder, file))
    #                 if data is not(False):
    #                     if data.shape[0] > max_length:
    #                         max_length = data.shape[0]
    # print(max_length)
    # Make the arrays
    X = np.empty((0, max_length, 300))
    Y = np.empty((0,), int)
    print("----------")
    for folder in os.listdir(folder_path):
        print(folder)
        if folder != ".DS_Store":
            
            i=0
            for file in os.listdir(os.path.join(folder_path, folder)):
                i+=1
                if file != ".DS_Store": 
                    data = process_csv_file(os.path.join(folder_path, folder, file))
                    if data is not(False):
                        data = set_padding(data, max_length)
                        X = np.append(X, data, axis=0)
                        Y = np.append(Y, labels[folder])
                        if (i% 50 == 0):
                            print(i)
                if i==5:
                    break

          
    print(total, max_length)
    return X, Y

In [50]:
folder_path_train = "/Tmp/linxinle/ASL-data/Database-augmented/train"
# folder_path_train="/media/makishima/DATA1/Personnel/Other learning/Programming/Personal_projects/ASL_Language_translation/training_models/mediapipe/data_25_labels_augmentation/train/"
labels= {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_train, Y_train = prepare_data(folder_path_train, labels)

----------
dog
door
coffee
milk
0 211


In [24]:
# folder_path_test = "/Tmp/ASL-data/Database-augmented/test"
folder_path_test = "/Tmp/linxinle/ASL-data/Database-augmented/test"

labels = {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_test, Y_test = prepare_data(folder_path_test, labels, train=False)

----------
dog
door
coffee
milk
0 211


In [25]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
], padding="post"
)
# print(X[0][1])
print(((padded_inputs)))

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [26]:
# Y = np.expand_dims(Y, axis=0)
# Y= Y.T


In [27]:
# shuffle data
from sklearn.utils import shuffle

X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_val,Y_val = shuffle(X_test, Y_test, random_state=0)
X_train, Y_train

(array([[[ 0.13714732,  0.21640315, -0.76089728, ...,  0.03080839,
           0.        ,  0.        ],
         [ 0.13900962,  0.21570578, -0.70383424, ...,  0.02660934,
           0.        ,  0.        ],
         [ 0.14016245,  0.21600241, -0.68726504, ...,  0.0246705 ,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],
 
        [[ 0.03935957, -0.55007347, -0.82141459, ..., -0.9058095 ,
           0.        ,  0.        ],
         [ 0.03496937, -0.55671575, -0.88811451, ..., -0.91157166,
           0.        ,  0.        ],
         [ 0.22295145, -0.25609224, -0.88870674, ..., -0.60995682,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.       

In [28]:
X_train.shape, Y_train.shape

((20, 211, 300), (20,))

In [29]:
x_shape = X_train.shape

In [30]:
from tensorflow.keras import layers
masking_input = []
for i in range(x_shape[0]):
    masking_input.append(X_train[i].T[0])

masking_input = np.array(masking_input)

masking_input.shape

(20, 211)

In [31]:
embedding = layers.Embedding(input_dim=x_shape[1], output_dim=x_shape[1], mask_zero=True)
masked_output = embedding(masking_input)

print(masked_output._keras_mask)
masked_output
# masking_layer = layers.Masking()
# # Simulate the embedding lookup by expanding the 2D input to 3D,
# # with embedding dimension of 10.
# unmasked_embedding = tf.cast(
#     tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
# )

# masked_embedding = masking_layer(unmasked_embedding)
# print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(20, 211), dtype=bool)


2023-06-19 18:43:38.595080: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-19 18:43:38.595356: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-19 18:43:38.595543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

<tf.Tensor: shape=(20, 211, 211), dtype=float32, numpy=
array([[[ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343],
        [ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343],
        [ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343],
        ...,
        [ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343],
        [ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343],
        [ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343]],

       [[ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343],
        [ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343],
        [ 0.0204925 , -0.02638774, -0.04027992, ...,  0.01537653,
         -0.0169578 , -0.02144343],
        ...

In [36]:
#LSTM model 
from tensorflow.keras import layers
x_shape = (457, 211, 300)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(211, 300)))
# model_lstm.add(Dropout(0.2))
# model_lstm.add(LSTM(128, return_sequences=True))
# model_lstm.add(Dropout(0.2))
# model_lstm.add(LSTM(64))
model_lstm.add(Flatten())
# model_lstm.add(Dense(128, activation="relu"))
# model_lstm.add(Dense(64, activation="relu"))
model_lstm.add(Dense(4, activation='softmax'))


2023-06-19 18:45:01.867637: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-19 18:45:01.868345: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-19 18:45:01.869086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [37]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    # "/Tmp/shariffa/LSTM-4-labels.{epoch:02d}-{val_accuracy:.2f}",
    "/Tmp/linxinle/models/LSTM-4-labels.{epoch:02d}-{val_accuracy:.2f}",

    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

In [38]:
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model_lstm.fit(X_train, Y_train, epochs=50, validation_data=(X_val, Y_val), callbacks=[checkpoint])

Epoch 1/50


2023-06-19 18:45:06.458525: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-19 18:45:06.459489: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-19 18:45:06.460176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_1/lstm_3/PartitionedCall]] [Op:__inference_train_function_12073]

In [ ]:
inputs = layers.Input(shape=(x_shape[1], x_shape[2]))
x = layers.GRU(64)(inputs, mask=masked_output._keras_mask)
# x = layers.LSTM(64)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model_lstm2= tf.keras.Model(inputs, outputs)
model_lstm2.summary()

In [ ]:
model_lstm2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm2.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)